### Import dataset

In [5]:
import pandas as pd
import numpy as np
# import category_encoders as ce
# import copy
# import polars as pl

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import matplotlib
import seaborn as sns
# import plotly.express as px
# %matplotlib inline
# matplotlib.rcParams['font.family'] = 'Malgun Gothic' # 한글 패치
# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile
from sklearn.decomposition import PCA

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling
# from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, VotingRegressor
from sklearn.ensemble import StackingClassifier, StackingRegressor
# from sklearn.base import ClassifierMixin

# CatBoost
# from catboost import CatBoostRegressor

# PyTorch
import torch
import torch.nn.init as init
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss,mean_squared_error
import sklearn

# Utility
import os
import time
import datetime # ⚠️2019년 12월30일과 31일의 week of year가 1인 오류가 있음
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations,product
from scipy.stats.mstats import gmean
import holidays

# from bayes_opt import BayesianOptimization
# from num2words import num2words
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

pd.set_option('display.max_row',None)
pd.set_option('display.max_column',None)

### Setting universal random_state
np.random.seed(142)
random.seed(142)
sklearn.utils.check_random_state(142)
torch.manual_seed(142)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False

try:
    import google.colab
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Note: not using Google CoLab
Using device: cuda


### Merge dataset

In [6]:
class PrepareData():
    def __init__(self,stock_fp,news_fp):
        self.stock_filepath = stock_fp
        self.news_filepath = news_fp
        self.topic_classes = ['CEO', 'CFO', 'Layoffs', 'Political', 'PressRelease', 'Undefined',
       'cramer', 'earnings', 'gold', 'manda', 'paylimitwall', 'paywall',
       'product', 'recession', 'tanalysis'] # undefined의 class가 5

    def load_data(self):
        stock = pd.read_csv(self.stock_filepath,index_col=0)
        news = pd.read_csv(self.news_filepath,index_col=0)
        ### parse date manually
        stock['Date'] = pd.to_datetime(stock['Date'])
        news['date'] = pd.to_datetime(news['date'])
        return stock, news

    def merging(self, stock, news):
        ### fill na value of PINS column
#         stock['PINS'] = stock['PINS'].fillna(stock['PINS'].iloc[75])

        ### drop 'news_id' column
#         news = news.drop(columns=['news_id'])

        ### add date range from 18.01.02 to 18.12.31
        temp_range = pd.DataFrame(dict(zip(stock.columns,[pd.date_range(start='2018-01-02',end='2018-12-31'),
                                0,0,0,0, # 4
                                0,0,0,0,
                                0,0,0,0,
                                0,0,0,0, # 16
                                0,0,0,0,
                                0,0,0,0,
                                0,0,0,0, # 28
                                0,0,0,0,
                                0,0,0,0, # 36
                                0 # 37
                            ])))
        stock_inc = pd.concat([temp_range,stock],axis=0)

        ### merge stock_inc and news
        # left = stock_inc
        # on = date
        # how = left
        # rename 'date' to 'Date' of news df
        news = news.rename(columns={'date':'Date'})
        merged = pd.merge(left=stock_inc,right=news,on='Date',how='left')

        ### Cut before 2018-02-13
        merged = merged[42:].reset_index(drop=True)

        # fill na with latest non-null values
        columns_to_fill = ['source_name', 'topics', 'rank_score',
                        'sentiment_Negative','sentiment_Neutral',
                        'sentiment_Positive', 'type_Article', 'type_Video']
        merged_fillna = merged.copy()
        for column in columns_to_fill:
            merged_fillna[column].fillna(method='ffill',inplace=True)

        ### add moving average to sentiments
        ma_nums = [5,60,120]
        def mode_window(window):
            return window.mode().iloc[0] if not len(window.mode())==0 else None
        for num in ma_nums:
            merged_fillna[f'{num}MA_sent_Neg']=merged_fillna['sentiment_Negative'].rolling(
            window=num).mean()
            merged_fillna[f'{num}MA_sent_Neu']=merged_fillna['sentiment_Neutral'].rolling(
            window=num).mean()
            merged_fillna[f'{num}MA_sent_Pos']=merged_fillna['sentiment_Positive'].rolling(
            window=num).mean()
        ### add moving mode to sentiments
        for num in ma_nums:
            merged_fillna[f'{num}MM_sent_Neg']=merged_fillna['sentiment_Negative'].rolling(
            window=num).apply(mode_window)
            merged_fillna[f'{num}MM_sent_Neu']=merged_fillna['sentiment_Neutral'].rolling(
            window=num).apply(mode_window)
            merged_fillna[f'{num}MM_sent_Pos']=merged_fillna['sentiment_Positive'].rolling(
            window=num).apply(mode_window)
        ### adding moving mode to topics
        for num in ma_nums:
            merged_fillna[f'{num}MM_topics']=merged_fillna['topics'].rolling(
            window=num).apply(mode_window)

        ### drop before 2019-01-02
        total_df = merged_fillna.iloc[322:]
        total_df = total_df.reset_index(drop=True)

        ### drop unnecessaray columns
        drop_cols = ['source_name','topics','rank_score',
                    'sentiment_Negative','sentiment_Neutral',
                    'sentiment_Positive','type_Article','type_Video']
        total_df = total_df.drop(columns=drop_cols)

        return total_df

In [7]:
stock_filepath = '../../data/stock_price/netflix_60.csv' # 각자 파일 경로 설정
news_filepath = '../../data/scraping/news_processed_filtered_2.csv'
# stock_filepath = './drive/MyDrive/Colab Notebooks/data/bitamin_mini_project/netflix_60.csv'
# news_filepath = './drive/MyDrive/Colab Notebooks/data/bitamin_mini_project/news_processed_filtered_2.csv'
loader = PrepareData(stock_filepath, news_filepath)
stock_df, news_df=loader.load_data() # >> 감성분석 미포함으로 모델 돌릴 땐 stock_df 바로 사용하면 됨
total_df = loader.merging(stock=stock_df, news=news_df) # 주식데이터셋에 감성분석,토픽 포함시킨 전체 데이터셋

stock_df.index = stock_df["Date"]
stock_df.drop(columns = "Date", inplace = True)
stock_df["PINS"].fillna(24.99, inplace = True)
total_df.index = total_df["Date"]
total_df.drop(columns = "Date", inplace = True)
total_df["PINS"].fillna(24.99, inplace = True)

In [8]:
result_df = pd.DataFrame({
    'data':[0],
    'target':[0],
    'seq_size':[0],
    'pred_size':[0],
    'batch_size':[0],
    'hidden_size':[0],
    'best_val_loss':[0],
    'mean_error_ratio':[0]
})
result_df

,data,target,seq_size,pred_size,batch_size,hidden_size,best_val_loss,mean_error_ratio
0,0,0,0,0,0,0,0,0


In [5]:
datanames = ['total_df','stock_df'] # stock_df / total_df
TARGETs = ["Close","1d_ROC"]                      # "Close" / "1d_ROC"
SEQ_SIZEs = [30,60,120]                         # 30 / 60 / 120
PRED_SIZE = 10
BATCH_SIZEs = [1,4,8]                        # 1 / 4 / 8
HIDDEN_SIZEs = [64,128]                   # 64 / 128

for dataname,TARGET,SEQ_SIZE,BATCH_SIZE,HIDDEN_SIZE in tqdm(product(
    datanames,TARGETs,SEQ_SIZEs,BATCH_SIZEs,HIDDEN_SIZEs)):
    ### Choose dataset & Hyperparameter setting
    if datanames=='total_df':
        data = total_df
    else:
        data = stock_df
#     TARGET = "1d_ROC"                      # "Close" / "1d_ROC"
#     SEQ_SIZE = 60                         # 30 / 60 / 120
#     PRED_SIZE = 10
#     BATCH_SIZE = 8                        # 1 / 4 / 8
#     HIDDEN_SIZE = 64                      # 64 / 128
    EPOCHS = 1000
    
    filename = f'{dataname}_{TARGET}_seq{SEQ_SIZE}_batch{BATCH_SIZE}_hidden{HIDDEN_SIZE}'

    print(f'🔸START - {filename}🔸')
    ### Make train datset

    def split_xy(dataset, time_steps, y_column):
        x, y = list(), list()
        for i in range(len(dataset)):
            x_end_number = i + time_steps
            y_end_number = x_end_number + y_column

            if y_end_number > len(dataset):
                break
            tmp_x = dataset.iloc[i:x_end_number, :]  # Adjusted for Pandas
            tmp_y = dataset.iloc[x_end_number:y_end_number, :].loc[:, TARGET]
            x.append(tmp_x.values)  # Convert to numpy array
            y.append(tmp_y.values)  # Convert to numpy array

        return np.array(x), np.array(y)

    X, y = split_xy(data, SEQ_SIZE, PRED_SIZE)
#     print(X[0,:],"\n", y[0])
#     print("X size : ", X.shape)
#     print("y size : ", y.shape)

    ### Define X_test

    X_test = data.tail(SEQ_SIZE).values.reshape(1, SEQ_SIZE, data.shape[1])
#     print(X_test)
#     print("X_test size : ", X_test.shape)

    ### Standardization

    from sklearn.preprocessing import StandardScaler

    X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    X = X.reshape(X.shape[0], SEQ_SIZE, data.shape[1])

    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2])
    X_test = scaler.transform(X_test)
    X_test = X_test.reshape(X_test.shape[0], SEQ_SIZE, data.shape[1])

#     print("X size : ", X.shape)
#     print("X_test size : ", X_test.shape)

    ### Split train-validation dataset

    # to DataLoader
    import torch
    from torch.utils.data import Dataset, DataLoader, TensorDataset
    from sklearn.model_selection import train_test_split

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state = 1, test_size = 0.2)

    # to tensor
    X_train = torch.tensor(X_train.astype(np.float32), dtype = torch.float32)
    X_valid = torch.tensor(X_valid.astype(np.float32), dtype = torch.float32)
    y_train = torch.tensor(y_train.astype(np.float32), dtype = torch.float32)
    y_valid = torch.tensor(y_valid.astype(np.float32), dtype = torch.float32)

    # to DataLoader
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size = BATCH_SIZE, shuffle = True)
    val_loader = DataLoader(TensorDataset(X_valid, y_valid), batch_size = BATCH_SIZE, shuffle = False)

    ### Initialize Model
    try:
        for param in model.parameters():
            if param.requires_grad:
                if len(param.shape) > 1:
                    init.xavier_uniform_(param)
                else:
                    init.zeros_(param)
    except:
        try:
            model.reset_parameters()
        except:
            pass

    ### Modeling

    import copy
    class EarlyStopping:
        def __init__(self, patience = 5, min_delta = 0, restore_best_weights = True):
            self.patience = patience
            self.min_delta = min_delta
            self.restore_best_weights = restore_best_weights
            self.best_model = None
            self.best_loss = None
            self.counter = 0
            self.status = ""

        def __call__(self, model, val_loss):
            if self.best_loss is None:
                self.best_loss = val_loss
                self.best_model = copy.deepcopy(model.state_dict())
            elif self.best_loss - val_loss >= self.min_delta:
                self.best_model = copy.deepcopy(model.state_dict())
                self.best_loss = val_loss
                self.counter = 0
                self.status = f"Improvement found, counter reset to {self.counter}"
            else:
                self.counter += 1
                self.status = f"No improvement in the last {self.counter} epochs"
                if self.counter >= self.patience:
                    self.status = f"Early stopping triggered after {self.counter} epochs."
                    if self.restore_best_weights:
                        model.load_state_dict(self.best_model)
                    return True
            return False

    # Positional Encoding for Transformer
    class PositionalEncoding(nn.Module):
        def __init__(self, d_model, dropout=0.1, max_len=1000):
            super(PositionalEncoding, self).__init__()
            self.dropout = nn.Dropout(p=dropout)

            pe = torch.zeros(max_len, d_model)
            position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
            div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
            pe[:, 0::2] = torch.sin(position * div_term)
            pe[:, 1::2] = torch.cos(position * div_term)
            pe = pe.unsqueeze(0).transpose(0, 1)
            self.register_buffer('pe', pe)

        def forward(self, x):
            x = x + self.pe[:x.size(0), :]
            return self.dropout(x)

    # Model definition using Transformer
    class TransformerModel(nn.Module):
        def __init__(self, input_dim, d_model, nhead=4, num_layers=2, dropout=0.2, output_size=10):
            super(TransformerModel, self).__init__()

            self.encoder = nn.Linear(input_dim, d_model,bias=True)
            self.pos_encoder = PositionalEncoding(d_model, dropout)
            encoder_layers = nn.TransformerEncoderLayer(d_model, nhead,bias=True)
            self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
            self.decoder = nn.Linear(d_model, output_size,bias=True)

        def forward(self, x):
            x = self.encoder(x)
            x = self.pos_encoder(x)
            x = self.transformer_encoder(x)
            x = self.decoder(x[:, -1, :])
            return x

    class RMSELoss(nn.Module):
        def __init__(self):
            super(RMSELoss,self).__init__()
            self.mse = nn.MSELoss()

        def forward(self,yhat,y):
            return torch.sqrt(self.mse(yhat,y))

    model = TransformerModel(X_train.shape[2],
                            d_model = HIDDEN_SIZE,
                            output_size = PRED_SIZE).to(device)
    criterion = RMSELoss()
    optimizer = optim.Adam(model.parameters(), lr = 0.001)
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor = 0.1, patience = 20, verbose = True)

    ### RUN!!

    epoch_counter = 0
    patience = 30
    best_loss = float('inf')
    done = False
    es = EarlyStopping(patience=patience)
    tr_losses_fp, val_losses_fp = [],[]

    while not done and epoch_counter<EPOCHS:
        epoch_counter+=1

        # train
        model.train()
        train_losses = []
        for batch in train_loader:
            x_batch, y_batch = batch
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output,y_batch)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        train_loss = np.mean(train_losses)
        tr_losses_fp.append(train_loss)

        # validation
        model.eval()
        val_losses = []
        with torch.no_grad():
            for batch in val_loader:
                x_batch, y_batch = batch
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                output = model(x_batch)
                loss = criterion(output, y_batch)
                val_losses.append(loss.item())
        val_loss = np.mean(val_losses)
        val_losses_fp.append(val_loss)
        scheduler.step(val_loss)

        if es(model, val_loss):
            done = True

        if val_loss < best_loss:
            best_loss = val_loss

        if epoch_counter%20 == 0:
            print(f"Epoch {epoch_counter}/{EPOCHS}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

    print(f"Best validation loss : {best_loss}")

    ### Visualize train-validation loss

    plt.plot(range(len(tr_losses_fp)),tr_losses_fp,color='blue',label='train_loss')
    plt.plot(range(len(val_losses_fp)),val_losses_fp,color='red',label='val_loss')
    plt.legend()
    # plt.show()
    plt.savefig(f'../../plots/train_val_loss_2/{filename}.png')
    plt.clf()


    ### Prediction

    # evaluation
    model.eval()
    with torch.no_grad():
        X_test = torch.tensor(X_test.astype(np.float32), dtype = torch.float32,device=device)
        pred = model(X_test)

    pred = pred.to('cpu').detach().numpy()
    # print(pred)

    if TARGET == "1d_ROC" :
        endPrice = data['Close'].iloc[-1]
        pred_close = []

        for i in pred[0] :
            endPrice = endPrice + endPrice*0.01*i
            pred_close.append(endPrice)

        pred = np.array(pred_close).reshape(1, PRED_SIZE)
    #     pred
    else :
        pass

    # pred_length = len(np.reshape(pred, (-1)))
    # pred_indices = list(range(pred_length))
    # plt.plot(pred_indices, np.reshape(pred, (-1)), color='red', alpha=0.6, label='Prediction')
    # plt.legend()

    # plt.show()

    ### Evaluation

    # Label
    testfile = '../../data/test.csv'
    label = pd.read_csv(testfile) # 각자 test.csv 파일 경로
    label = np.array(label.head(PRED_SIZE)["Close Price"])

    # Prediction
    pred = np.array(pred).reshape(PRED_SIZE)

    # 날짜 데이터
    period = pd.read_csv(testfile)["Date"]
    period = [d for d in period.head(PRED_SIZE)]

    # 오차율 계산
    error_rate = np.abs((label - pred) / label) * 100

    # 시각화
    plt.figure(figsize=(12, 6))
    plt.plot(period, label, marker='o', color='blue', label='Actual Close Price')
    plt.plot(period, pred, marker='x', color='red', linestyle='--', label='Predicted Close Price')

    # 오차율을 각 포인트에 텍스트로 표시
    for date, lbl, prd, err in zip(period, label, pred, error_rate):
        plt.text(date, prd, f'{err:.2f}%', color='black', ha='right', va='bottom')

    plt.xticks(rotation = 45)  # 날짜 레이블 회전
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title(f'model = LSTM, data = {dataname}, target = {TARGET}, seq_size = {SEQ_SIZE}, pred_size = {PRED_SIZE}, batch_size = {BATCH_SIZE}, model_size = {HIDDEN_SIZE}')
    plt.legend()
    plt.tight_layout()  # 레이아웃 조정
    # plt.show()
    plt.savefig(f'../../plots/results_2/{filename}.png')
    plt.clf()

    # 오차율을 출력
    error_rate_dict = dict(zip(period, error_rate))
    # error_rate_dict

    # 평균 오차율 계산
    average_error_rate = np.mean(error_rate)
    print(f"Average Error Rate: {average_error_rate:.2f}%")

    ### Save results to a DataFrame
    temp_df = pd.DataFrame({
        'data':[dataname],
        'target':[TARGET],
        'seq_size':[SEQ_SIZE],
        'pred_size':[PRED_SIZE],
        'batch_size':[BATCH_SIZE],
        'hidden_size':[HIDDEN_SIZE],
        'best_val_loss':[best_loss],
        'mean_error_ratio':[average_error_rate]
    })

    result_df = pd.concat([result_df,temp_df],axis=0)
print('🔸END🔸')

0it [00:00, ?it/s]

🔸START - total_df_Close_seq30_batch1_hidden64🔸
Epoch 20/1000, Train Loss: 26.7503, Validation Loss: 23.8937
Epoch 40/1000, Train Loss: 21.7205, Validation Loss: 19.8776
Epoch 00051: reducing learning rate of group 0 to 1.0000e-04.
Epoch 60/1000, Train Loss: 19.4973, Validation Loss: 18.1150
Epoch 80/1000, Train Loss: 18.6441, Validation Loss: 17.4605
Epoch 00090: reducing learning rate of group 0 to 1.0000e-05.
Best validation loss : 17.283372642563993


1it [16:38, 998.69s/it]

Average Error Rate: 7.70%
🔸START - total_df_Close_seq30_batch1_hidden128🔸
Epoch 20/1000, Train Loss: 25.7851, Validation Loss: 20.4350
Epoch 40/1000, Train Loss: 21.4716, Validation Loss: 22.4750
Epoch 00049: reducing learning rate of group 0 to 1.0000e-04.
Epoch 60/1000, Train Loss: 18.9022, Validation Loss: 19.1728
Epoch 80/1000, Train Loss: 18.4786, Validation Loss: 17.9376
Epoch 100/1000, Train Loss: 18.1583, Validation Loss: 18.4540
Epoch 00104: reducing learning rate of group 0 to 1.0000e-05.
Best validation loss : 17.014774048914674


2it [35:13, 1066.87s/it]

...

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

In [3]:
# result_df = result_df.iloc[1:].reset_index(drop=True)
result_df.to_csv('../../data/final_results_0219.csv')

NameError: name 'result_df' is not defined

In [9]:
result_df = pd.read_csv('../../data/final_results_0219.csv',index_col=0)
result_df.head(2)

,data,target,seq_size,pred_size,batch_size,hidden_size,best_val_loss,mean_error_ratio
0,total_df,Close,30,10,1,64,17.283373,7.700228
1,total_df,Close,30,10,1,128,17.014774,3.396892


In [10]:
result_df.sort_values(by=['mean_error_ratio'])

,data,target,seq_size,pred_size,batch_size,hidden_size,best_val_loss,mean_error_ratio
51,stock_df,Close,120,10,4,128,33.504185,1.431333
16,total_df,Close,120,10,8,64,16.852619,1.431409
24,total_df,1d_ROC,60,10,1,64,2.548527,1.668213
63,stock_df,1d_ROC,60,10,4,128,2.785007,1.697323
66,stock_df,1d_ROC,120,10,1,64,2.492974,1.729040
14,total_df,Close,120,10,4,64,14.167484,1.743051
18,total_df,1d_ROC,30,10,1,64,2.306839,1.750611
54,stock_df,1d_ROC,30,10,1,64,2.303385,1.765025
21,total_df,1d_ROC,30,10,4,128,2.558942,1.831137
8,total_df,Close,60,10,4,64,18.655700,1.908761
